In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

In [2]:
#first we need a list of valid words.
#get the dictionary of words from some url like this.

import urllib3
http=urllib3.PoolManager()
r=http.request('GET','https://raw.githubusercontent.com/dwyl/english-words/master/words.txt',preload_content=False)

In [3]:
with open("words.txt",'wb') as out:
    data=r.read()
    out.write(data)
r.release_conn()

4862992

In [4]:
#lets clean up and remove line endings from each word in the file.

word_list=open('words.txt').readlines()
word_list=map(str.strip, word_list)

In [5]:
#we want words of length 3,so lets select those words.
#we will also eliminate words which start with upper-case or contain alpha-numeric characters.
#finally we will lower-case everything.

word_list=[word for word in word_list if len(word)==3]
word_list=[word for word in word_list if word[0].islower()]
word_list=[word for word in word_list if word.isalpha()]
word_list=list(map(str.lower,word_list))
len(word_list)

1018

In [6]:
word_list

['aah',
 'aal',
 'abb',
 'abd',
 'aby',
 'abl',
 'abn',
 'abr',
 'abt',
 'abv',
 'acy',
 'ade',
 'ady',
 'adj',
 'ado',
 'adv',
 'adz',
 'aeq',
 'aer',
 'afd',
 'aff',
 'aga',
 'age',
 'agy',
 'ago',
 'ahi',
 'aho',
 'ahs',
 'ahu',
 'aye',
 'aik',
 'ail',
 'ays',
 'ait',
 'ayu',
 'ake',
 'ako',
 'aku',
 'ale',
 'aly',
 'alk',
 'all',
 'aln',
 'alt',
 'alw',
 'ana',
 'and',
 'ane',
 'ans',
 'ant',
 'aob',
 'aor',
 'aph',
 'apx',
 'arb',
 'ard',
 'ary',
 'arn',
 'arr',
 'art',
 'arx',
 'asb',
 'ase',
 'ast',
 'ate',
 'aud',
 'auf',
 'auh',
 'aul',
 'avg',
 'avn',
 'awa',
 'awd',
 'awe',
 'awm',
 'awn',
 'azo',
 'bad',
 'bah',
 'baw',
 'bbs',
 'bcf',
 'bdl',
 'bec',
 'beg',
 'bey',
 'bet',
 'bhd',
 'bye',
 'big',
 'bin',
 'bio',
 'byp',
 'bys',
 'biz',
 'bkg',
 'bks',
 'bkt',
 'bld',
 'ble',
 'blk',
 'blo',
 'boa',
 'boe',
 'bog',
 'boh',
 'boo',
 'bpt',
 'brr',
 'bsh',
 'buy',
 'bun',
 'buz',
 'bvt',
 'bxs',
 'cag',
 'cay',
 'caw',
 'ccm',
 'cdg',
 'cee',
 'cep',
 'cfh',
 'cfm',
 'cfs',


Each of these 1018 words will become a node in our graph and we will create edges connecting the node associated with each pair of words which differ by only one letter.

In [7]:
import numpy as np
word_list=np.asarray(word_list)
word_list.dtype
word_list.sort()

dtype('<U3')

In [8]:
word_list.itemsize

12

We have an array where each entry is three unicode characters long. We did like to find all pairs where exactly one character is different. We will start by converting each word to a three dimensional vector.

In [9]:
word_bytes=np.ndarray((word_list.size,word_list.itemsize),dtype='uint8',buffer=word_list.data)

#Each unicode character is four bytes long.
#We only need first byte 
#we know there are three characters per word

word_bytes.shape

(1018, 12)

In [10]:
word_bytes

array([[ 97,   0,   0, ...,   0,   0,   0],
       [ 97,   0,   0, ...,   0,   0,   0],
       [ 97,   0,   0, ...,   0,   0,   0],
       ...,
       [122,   0,   0, ...,   0,   0,   0],
       [122,   0,   0, ...,   0,   0,   0],
       [122,   0,   0, ...,   0,   0,   0]], dtype=uint8)

In [11]:
word_bytes=word_bytes[:,::word_list.itemsize//3]
word_bytes.shape
word_list[0:4]
word_bytes[0:4]

(1018, 3)

array(['aah', 'aal', 'abb', 'abd'], dtype='<U3')

array([[ 97,  97, 104],
       [ 97,  97, 108],
       [ 97,  98,  98],
       [ 97,  98, 100]], dtype=uint8)

Now we will use the Hamming Distance between each point to determine which pair of words are connected.The Hamming distance measures the fraction of entries between two vectors which differ: any two words with a Hamming distance equal to 1/N, where N is the number of letters, are connected in the word ladder.

In [12]:
from scipy.spatial.distance import pdist

#pdist: pairwise distance between objects in n-D space.

hamming_dist=pdist(word_bytes,metric='hamming')
hamming_dist.shape
(1018*1018-1018)//2

(517653,)

517653

In [13]:
from scipy.spatial.distance import squareform

#Squareform: converts a vector-form distance vector to a squareform distance matrix
squareform(hamming_dist)[0:4,0:4]

#there are three characters in each word
mat=squareform(hamming_dist <1.5/3)
mat.shape
mat[0:4,0:4]

array([[0.        , 0.33333333, 0.66666667, 0.66666667],
       [0.33333333, 0.        , 0.66666667, 0.66666667],
       [0.66666667, 0.66666667, 0.        , 0.33333333],
       [0.66666667, 0.66666667, 0.33333333, 0.        ]])

(1018, 1018)

array([[False,  True, False, False],
       [ True, False, False, False],
       [False, False, False,  True],
       [False, False,  True, False]])

In [14]:
from scipy.sparse import csr_matrix
graph=csr_matrix(mat)

Now that our graph is set up, we will use a sortest path search to find the path between any to words in the graph.All we need is to find the shortest path between these two indices in the graph. We'll use Dijkstra's algorithm, because it allows us to find the path for just one node.

In [15]:
i1=word_list.searchsorted('and')
i2=word_list.searchsorted('dog')
word_list[i1]
word_list[i2]

'and'

'dog'

In [16]:
from scipy.sparse.csgraph import dijkstra
distances, predecessors = dijkstra(graph,indices=i1,return_predecessors=True)
distances[i2]

5.0

So we see that the shortest path between "and" and "dog" contains only five steps. We can use the predecessors returned by the algorithm to reconstruct this path.

In [17]:
path=[]
i=i2
while i != i1:
    path.append(word_list[i])
    i=predecessors[i]
path.append(word_list[i1])
path[::-1]

['and', 'aud', 'mud', 'mug', 'dug', 'dog']

## Thats all about creating a word ladder game.

Lets do some more fancier things:
- Are there three-letter words which are not linked in a word ladder?<br>
this is a question of connected components in the graph.

In [18]:
from scipy.sparse.csgraph import connected_components
N_components, component_list = connected_components(graph)
N_components
component_list.shape
component_list[0:5]

9

(1018,)

array([0, 0, 0, 0, 0])

In [19]:
[np.sum(component_list == i) for i in range(N_components)]

[1010, 1, 1, 1, 1, 1, 1, 1, 1]

In [20]:
[list(word_list[np.where(component_list == i)]) for i in range(1,N_components)]
#these are all the three-letter words which do not connect to others via a word ladder.

[['bcf'], ['bdl'], ['epi'], ['mmf'], ['sml'], ['wjc'], ['xcl'], ['xyz']]

- Which words are maximally separated? Which two words take the most links to connect?<br>
- We can determine this by computing the matrix of all shotest paths.
- Note that by convention, the distance between two non-connected points is reported to be infinity, so we will need to remove these before finding the maximum.

In [21]:
distances, predecessors = dijkstra(graph, return_predecessors=True)
max_distance = np.max(distances[~np.isinf(distances)])
max_distance

i1,i2=np.where(distances == max_distance)
list(zip(word_list[i1],word_list[i2]))

11.0

[('dlr', 'hmm'),
 ('dlr', 'kmc'),
 ('hmm', 'dlr'),
 ('hmm', 'lbf'),
 ('hmm', 'rfz'),
 ('hmm', 'tfr'),
 ('kmc', 'dlr'),
 ('kmc', 'tfr'),
 ('lbf', 'hmm'),
 ('rfz', 'hmm'),
 ('tfr', 'hmm'),
 ('tfr', 'kmc')]

In [22]:
#list down all the long chains
for j,i in zip(i1,i2):
    path=[]
    while i !=j:
        path.append(word_list[i])
        i = predecessors[j,i]
    path.append(word_list[j])
    print(path[::-1])

['dlr', 'tlr', 'tlo', 'blo', 'boo', 'boh', 'bsh', 'ush', 'usu', 'umu', 'umm', 'hmm']
['dlr', 'tlr', 'tlo', 'blo', 'bld', 'bad', 'bah', 'pah', 'pph', 'kph', 'kpc', 'kmc']
['hmm', 'umm', 'umu', 'usu', 'ust', 'ugt', 'tgt', 'tot', 'too', 'tlo', 'tlr', 'dlr']
['hmm', 'umm', 'umu', 'usu', 'ust', 'ast', 'abt', 'abv', 'dbv', 'dbw', 'lbw', 'lbf']
['hmm', 'umm', 'ump', 'unp', 'uns', 'ons', 'ous', 'ouf', 'suf', 'suz', 'sfz', 'rfz']
['hmm', 'umm', 'umu', 'usu', 'ust', 'ugt', 'tgt', 'tot', 'too', 'tlo', 'tlr', 'tfr']
['kmc', 'kpc', 'kph', 'iph', 'ich', 'ick', 'ilk', 'blk', 'blo', 'tlo', 'tlr', 'dlr']
['kmc', 'kpc', 'kph', 'iph', 'ich', 'ick', 'ilk', 'blk', 'blo', 'tlo', 'tlr', 'tfr']
['lbf', 'lbw', 'dbw', 'daw', 'baw', 'bah', 'bsh', 'ush', 'usu', 'umu', 'umm', 'hmm']
['rfz', 'rfb', 'rhb', 'rho', 'sho', 'shi', 'ihi', 'imi', 'imu', 'umu', 'umm', 'hmm']
['tfr', 'tlr', 'tlo', 'blo', 'boo', 'boh', 'bsh', 'ush', 'usu', 'umu', 'umm', 'hmm']
['tfr', 'tlr', 'tlo', 'blo', 'bld', 'bad', 'bah', 'pah', 'pph', '